In [1]:
import pandas as pd
import numpy as np

import glob

from pathlib import Path
import csv

import os
import requests
import json

import quandl
#from iexfinance.stocks import Stock

from dotenv import load_dotenv
load_dotenv()

True

In [15]:
## Constants 

## Set start date variable - dataframes will be created starting from this date
start_date = '2020-01-01'
end_date = '2021-11-09'
default_date_range = '2y' ## Default Range for IEX functions - don't need more at the moment

In [3]:
## QUANDL/NASDAQ 
nsdq_api_key = os.environ.get('NASDAQ_API_KEY')
base_url_nsdq = 'https://data.nasdaq.com/api/v3/datasets/FINRA/'

# ## Test iexfinance.stocks library setup and stability - should not be needed for this file 
# iex_token = os.getenv("IEX_TOKEN")
# iex_api_ver = os.getenv("IEX_API_VERSION")
# iex_out_form = os.getenv("IEX_OUTPUT_FORMAT")

iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

real_token = iex_api_key
test_token = iex_test_api_key

base_url_iex = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

## IEX Status Test 
test_resp = requests.get(base_url_iex + 'status')
test_resp

<Response [200]>

In [12]:
## IEX Calls and Functions 

def get_chart(stock_ticker, chart_range=default_date_range,token=test_token):
    
    if token == test_token:
        resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?chartCloseOnly=true&token='+test_token)
        df = pd.DataFrame(resp_data.json())
    elif token == real_token:
        resp_data = requests.get(base_url_iex+'stock/'+stock_ticker+'/chart/'+chart_range+'?chartCloseOnly=true&token='+real_token)
        df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    return df

In [13]:
test_df = get_chart('GME','2y')

In [14]:
test_df

,close,volume,change,changePercent,changeOverTime
Date,,,,,
2019-11-11,6.30,3149553,0.000000,0.0000,0.000000
2019-11-12,6.04,1875079,-0.100176,-0.0172,-0.016944
2019-11-13,6.10,1610429,0.020821,0.0035,-0.013596
2019-11-14,6.02,1721149,0.041413,0.0067,-0.006847
2019-11-15,6.07,2260076,-0.100183,-0.0169,-0.023288
...,...,...,...,...,...
2021-11-02,215.47,3958313,6.997630,0.0357,34.310564
2021-11-03,225.19,11629039,11.456366,0.0550,36.170633
2021-11-04,220.54,2952792,-0.505885,-0.0022,35.476583


In [7]:
## I'm doing this at 1AM on Nov 10th and can't figure out why it won't give Nov 9th Data... 

## It's because IEX doesn't publish data until 4am... 

In [8]:
#resp_data = requests.get(sandbox_url+'stock/twtr/chart/5d?token='+test_token)
test_resp_data = requests.get(sandbox_url+'stock/gme/chart/2y?chartCloseOnly=true&token='+test_token)
## Use "&" symbol to seperate parameter requests, and to add token to end of request. 
test_resp_data

<Response [200]>

In [11]:
data = test_resp_data.json()
data = pd.DataFrame(data)
data.head()

,date,close,volume,change,changePercent,changeOverTime
0,2019-11-11,6.03,3184078,0.000000,0.0000,0.000000
1,2019-11-12,6.16,1899621,-0.100421,-0.0167,-0.016905
2,2019-11-13,6.04,1587308,0.020200,0.0035,-0.013721
3,2019-11-14,6.13,1714226,0.041737,0.0068,-0.006874
4,2019-11-15,6.17,2255449,-0.104659,-0.0170,-0.023346
